In [1]:
# !pip install Box2D
# !pip install box2d-py
# !pip install gym[all]
# !pip install gym[Box_2D]
# !pip install torc
# !pip install -U "ray[rllib]" torch
import pickle as pkl
import gym 
env = gym.make("LunarLander-v2")

In [2]:
import ray
import ray.rllib.agents.ppo as ppo
from ray.tune.logger import pretty_print

config = ppo.DEFAULT_CONFIG.copy()
config["num_gpus"] = 0
config["num_workers"] = 1
trainer = ppo.PPOTrainer(config=config, env="LunarLander-v2")

# Can optionally call trainer.restore(path) to load a checkpoint.

for i in range(2):
   # Perform one iteration of training the policy with PPO
   result = trainer.train()
   print(pretty_print(result))

   if i % 2 == 0:
       checkpoint = trainer.save()
       print("checkpoint saved at", checkpoint)

# Also, in case you have trained a model outside of ray/RLlib and have created
# an h5-file with weight values in it, e.g.
# my_keras_model_trained_outside_rllib.save_weights("model.h5")
# (see: https://keras.io/models/about-keras-models/)

# ... you can load the h5-weights into your Trainer's Policy's ModelV2
# (tf or torch) by doing:

# NOTE: In order for this to work, your (custom) model needs to implement
# the `import_from_h5` method.
# See https://github.com/ray-project/ray/blob/master/rllib/tests/test_model_imports.py
# for detailed examples for tf- and torch trainers/models.

 pid=8244) 2022-04-18 18:05:50,632	WARNING deprecation.py:45 -- DeprecationWarning: `rllib.env.remote_vector_env.RemoteVectorEnv` has been deprecated. Use `ray.rllib.env.remote_base_env.RemoteBaseEnv` instead. This will raise an error in the future!
2022-04-18 18:05:52,239	INFO trainable.py:125 -- Trainable.setup took 79.601 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.
2022-04-18 18:06:01,484	WARNING deprecation.py:45 -- DeprecationWarning: `slice` has been deprecated. Use `SampleBatch[start:stop]` instead. This will raise an error in the future!
2022-04-18 18:06:09,249	WARNING deprecation.py:45 -- DeprecationWarning: `clear_buffer` has been deprecated. Use `Filter.reset_buffer()` instead. This will raise an error in the future!


agent_timesteps_total: 4000
custom_metrics: {}
date: 2022-04-18_18-06-09
done: false
episode_len_mean: 92.11627906976744
episode_media: {}
episode_reward_max: 40.9455968622222
episode_reward_mean: -213.21258224279157
episode_reward_min: -434.9219179132078
episodes_this_iter: 43
episodes_total: 43
experiment_id: fe5a15e5aa1148abab32edf9fe87eff2
hostname: DESKTOP-UETR0GC
info:
  learner:
    default_policy:
      custom_metrics: {}
      learner_stats:
        cur_kl_coeff: 0.20000000298023224
        cur_lr: 4.999999873689376e-05
        entropy: 1.3766803741455078
        entropy_coeff: 0.0
        kl: 0.009528086520731449
        model: {}
        policy_loss: -0.0005107718170620501
        total_loss: 11731.158203125
        vf_explained_var: -0.0041581797413527966
        vf_loss: 11731.15625
  num_agent_steps_sampled: 4000
  num_agent_steps_trained: 4000
  num_steps_sampled: 4000
  num_steps_trained: 4000
  num_steps_trained_this_iter: 4000
iterations_since_restore: 1
node_ip: 127.

In [3]:
import ray
from ray import tune

analysis = tune.run(
    "PPO",
    metric="episode_reward_mean",
    mode="max",
    stop={"training_iteration": 2000},
    config={
        "env": "LunarLander-v2",
        "num_gpus": 0,
        #"num_workers": 1,
        "lr": tune.grid_search([0.01, 0.001, 0.0001]),
    },
    verbose=1,
)

2022-04-19 08:59:53,675	INFO tune.py:636 -- Total run time: 53607.19 seconds (53606.48 seconds for the tuning loop).


In [4]:
print("Best hyperparameters found were: ", analysis.best_config)

Best hyperparameters found were:  {'env': 'LunarLander-v2', 'num_gpus': 0, 'lr': 0.0001}


In [5]:
analysis.dataframe(metric="episode_reward_mean", mode="max")[['episode_reward_mean', 'episode_reward_max', 'config/lr']]

,episode_reward_mean,episode_reward_max,config/lr
0,-132.069360,26.204158,0.0100
1,200.376501,290.861195,0.0010
2,282.199921,317.006130,0.0001


In [6]:
# save analysis to file
with open("analysis_finalDFs_PPO.pkl", "wb") as f:
    pkl.dump(analysis.dataframe(), f)

with open("analysis_trialDFs_PPO.pkl", "wb") as f:
    pkl.dump(analysis.trial_dataframes, f)

with open("analysis_configs_PPO.pkl", "wb") as f:
    pkl.dump(analysis._configs, f)